In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
import random

from tools.classical import *
from tools.quantum import *

#TODO: Try with Trotterized ES to see if we get close to exact result
#TODO: Use 2nd order trotter

In [2]:
num_estimating_bits = 4
num_qubits = 2
T = 1
# For 2x2 matrix one qubit is enough
qr_sys = QuantumRegister(num_qubits, name="q")
# In QPE we use n ancillas to estimate n bits from the phase
qr_energy = QuantumRegister(num_estimating_bits, name="E") 
# For n ancillary qubit measurment we need n cllasical bits
cr_energy = ClassicalRegister(num_estimating_bits, name="c") # Create a quantum circuit
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)

In [3]:
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)

# get all combinations of coefficients
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz)).T.reshape(-1, 3)

for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeff, num_qubits=num_qubits)
    exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
    exact_spec = np.round(exact_spec, 3)

    if len(np.unique(exact_spec)) == len(exact_spec):
        print("Unique spectrum found: ", exact_spec)
        print("Coefficients: ", coeff)
        break

coeffs = coeff / 5.4
hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeffs, num_qubits=num_qubits)
spectrum, eigenstates = np.linalg.eigh(hamiltonian_qt)
print(f'Coeffs used: {coeffs} for spectrum {spectrum}')

eigenstate = eigenstates[1]
exact_energy = spectrum[1]
print(f'Energy to predict {spectrum[1]}')


Unique spectrum found:  [-5.4  1.4  1.8  2.2]
Coefficients:  [0.9 0.8 1. ]
Coeffs used: [0.16666667 0.14814815 0.18518519] for spectrum [-1.          0.25925926  0.33333333  0.40740741]
Energy to predict 0.2592592592592592


In [4]:
def trotter_step_circ(num_qubits: int, coeffs = [1, 1, 1]):
    trotter_step_circ = QuantumCircuit(qr_sys.size)

    step_size = Parameter('theta')
    for i in range(qr_sys.size):
        if i != num_qubits - 1:
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size, i, i + 1)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size, i, i + 1)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size, i, i + 1)
        if (i == num_qubits - 1):
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size, i, 0)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size, i, 0)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size, i, 0)
    return trotter_step_circ

def second_order_trotter_step_circ(num_qubits: int, coeffs = [1, 1, 1]): #TODO: TEST IT
    trotter_step_circ = QuantumCircuit(qr_sys.size)

    step_size = Parameter('theta')
    
    for i in range(qr_sys.size):
        if i != num_qubits - 1:
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size / 2, i, i + 1)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size / 2, i, i + 1)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size / 2, i, i + 1)
        if (i == num_qubits - 1):
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size / 2, i, 0)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size / 2, i, 0)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size / 2, i, 0)
    
    for i in range(qr_sys.size, 0, -1):
        if i != 0:
            trotter_step_circ.rzz(-2 * coeffs[0] * step_size / 2, i, i + 1)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size / 2, i, i + 1)
            trotter_step_circ.rxx(-2 * coeffs[2] * step_size / 2, i, i + 1)
        if (i == 0):
            trotter_step_circ.rzz(-2 * coeffs[0] * step_size / 2, i, qr_sys.size - 1)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size / 2, i, qr_sys.size - 1)
            trotter_step_circ.rxx(-2 * coeffs[2] * step_size / 2, i, qr_sys.size - 1)
    
    return trotter_step_circ

def trotter_circ_for_qpe(trotter_step_circ: QuantumCircuit, total_time: float, num_trotter_steps: int) -> QuantumCircuit:
    trotter_circ = QuantumCircuit(qr_sys.size)
    step_size = 2*np.pi * total_time / num_trotter_steps
    trotter_step_circ.assign_parameters([step_size], inplace=True)
    for _ in range(num_trotter_steps):
        trotter_circ.compose(trotter_step_circ, inplace=True)
        
    return trotter_circ

num_trotter_steps = 50
heisenberg_trott_circ = trotter_circ_for_qpe(trotter_step_circ(num_qubits, coeffs), T, num_trotter_steps)
cU = heisenberg_trott_circ.control(1)

In [5]:
initial_state = Statevector(eigenstate.reshape(2**num_qubits, 1))
circ.initialize(initial_state, qr_sys)
circ.h(qr_energy)

for n in range(qr_energy.size):
    for m in range(2**n):
        circ.compose(cU, qubits=[qr_energy[n], *list(qr_sys)], inplace=True)
        
qft_circ = QFT(num_estimating_bits, do_swaps=True, inverse=True)
circ.compose(qft_circ, qr_energy, inplace=True)
circ.measure(qr_energy, cr_energy)
tr_circuit = transpile(circ, basis_gates=['u3', 'cx'], optimization_level=3)

/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [6]:
backend = Aer.get_backend('qasm_simulator')
shots = 256
job = backend.run(tr_circuit, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)
phase_bits = max(counts, key=counts.get) # take the most often obtaned result
print(f'Max count was for phase bits {phase_bits} : {counts[phase_bits]} times')

{'1001': 2, '1111': 1, '1101': 1, '1010': 1, '0101': 47, '0110': 37, '0100': 61, '0001': 1, '0011': 1, '0111': 38, '1000': 4, '0010': 2, '0000': 60}
Max count was for phase bits 0100 : 61 times


In [7]:
phase = 0
for index, bit in enumerate((phase_bits)):  #! Bits are already in order
    phase += int(bit) / 2**(index + 1)
    
estimated_energy = phase / T
print(f'Exact energy: {exact_energy}')
print(f'Estimated energy: {estimated_energy}')

Exact energy: 0.2592592592592592
Estimated energy: 0.25
